In [2]:
import pandas as py
import keys

ModuleNotFoundError: No module named 'keys'

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://localhost:5433/<SQL-Challenge>')
connection = engine.connect()
